In [1]:
import numpy as np
import copy
import itertools
import random
import time
#import math

In [2]:
class Board(object):
    '''棋盘，默认是“井字棋”Tie Tac Toe，也可以作为五子棋的棋盘'''
    def __init__(self, height=3, width=3, ninline=3):
        self.width = width     #列
        self.height = height   #行 
        self.ninline = ninline    # 表示几个相同的棋子连成一线算作胜利
        
        #''' # M x N 矩阵，值是棋子类型，0空，1代表'x'，-1代表'o'; ''' 
        self.ox = np.zeros((self.height, self.width), dtype=int )
        
        #'''现棋盘空余可下位置集合如{(0,0), (0,1), }'''
        self._ox_availables = set( [(i,j) for i in range(self.height) for j in range(self.width)] )
        
        #'''棋盘棋局胜负标志，1代表'x'赢，-1代表'o'赢，0代表平局，2代表未结束'''
        self._ox_states = 2
                    

    def _nmatrixwinner(self,nm):
        '''计算方阵连线情况,1代表'x'连成一线，-1代表'o'连成一线，0代表没有
           把各行列的和组成集合，查找集合中是否存在3或-3，代表连成一行'''
        n_line = set(nm.sum(axis=1)) | set(nm.sum(axis=0))
        '''第一条对角线的和加入列表'''
        n_line.add(sum([nm[i, i] for i in range(self.ninline) ]))
        '''第二条对角线的和加入列表'''
        n_line.add(sum([nm[self.ninline-1-i,i] for i in range(self.ninline) ]))
    
        return -1 if -self.ninline in n_line else 1 if self.ninline in n_line else 0 
    
        
    def _checkwinner(self):
        '''检查棋盘棋局胜负，1代表'x'赢，-1代表'o'赢，0代表平局，2代表未结束'''
        
        #'''生成棋盘所有ninline维方阵，并判断方阵是否有连成一线的情况'''
        matrixlist = [self.ox[i:i+self.ninline, j:j+self.ninline] 
                      for i in range(self.height- self.ninline +1)
                        for j in range(self.width- self.ninline +1)
                     ]
    
        #'''如果出现ninline维方阵返回1或者-1，代表已经有了输赢，返回输赢结果'''
        for nm in matrixlist:
            if self._nmatrixwinner(nm) != 0:
                return self._nmatrixwinner(nm) 
        
        #'''所有ninline维方阵返回都是0，检查是否有可落子处，无则返回0代表平局，有则2代表未结束'''
        return 0 if not self._ox_availables else 2
          
    
    def board_now(self):
        return (self._ox_states, self._ox_availables)    #返回一个元组，第一个元素是输赢状态，第二个元素是剩余可落子位置的集合
    
    
    def board_update(self, player, location): 
        '''# player在location(x,y)处落子，更新棋盘，,更新可用位置，计算此时棋局胜负'''
        if location in self._ox_availables:
            self.ox[location] = 1 if player=='x' else -1
            self._ox_availables.remove( location )    
            self._ox_states = self._checkwinner()
            return 1    
        else:
            return 0     
                  
            
    def board_show(self):
        '''显示棋盘，参数是棋盘矩阵'''
        #'''打印列标号'''
        print("{0:>25}".format('x\y'), end='') # 5个字符
        for j in range(self.width):
            print("{0:^5}".format(j), end='') # 5个字符          
        print('\n')  # 换行,打印空行        
      
        #'''打印棋盘'''            
        for i in range(self.height):        #行数
            #'''打印行标号'''
            print("{0:>25}".format(i), end='') # 5个字符
            #'''打印该行棋盘'''    
            for j in range(self.width):
                if self.ox[i][j] == 0:    #未落子  5个字符
                    print("{0:^5}".format('+'), end='')
                elif self.ox[i][j] == 1:   # x 落子  5个字符
                    print("{0:^5}".format('X'), end='')
                elif self.ox[i][j] == -1:  # o 落子
                    print("{0:^5}".format('Q'), end='') 
            print('\n')     
            
            

In [25]:
def ttt_simu(ttt, playturn, ms=[]):
    '''根据给定棋盘现状和后续下法，返回棋盘最终状态，谁赢或者平局'''
    ox = copy.deepcopy(ttt)
    for i in range(len(ms)):
        ox.board_update(playturn, ms[i])    # 更新棋盘
        s, (v) = ox.board_now()   
        if s == 1:
            return len(ms)-i
        elif s == -1:
            return i-len(ms)       
        elif s == 2 :
            playturn = 'o' if playturn=='x' else 'x'
    return 0
    

def ai_montecarlo_search(ox, playturn, mounts=1000, timelast=3):
    timebegin = time.time()
    s, (v) = ox.board_now()
    move_value = {x:0 for x in v}
    for count in range(mounts):
        mv = []
        v_setlist = list(v)
        for i in range(len(v)):
            mv.append(v_setlist.pop(random.randint(0,len(v_setlist)-1)))
            
        move_value[mv[0]] += ttt_simu(ox,playturn, mv)
        if (time.time()-timebegin) >= 3 : break
        
    if playturn == 'o':
        minvalue = move_value[min(move_value, key=move_value.get)]
        mvlist = [k for k,kv in move_value.items() if kv==minvalue ]
    if playturn == 'x':
        maxvalue = move_value[max(move_value, key=move_value.get)]
        mvlist = [k for k,kv in move_value.items() if kv==maxvalue ]

    return mvlist[random.randint(0,len(mvlist)-1)] #返回元组(x,y)
    
    
def ai_full_search(ox, playturn):
    '''AI走一步，但每走一步都穷尽所有走法，找到最好的,'''
    #timebegin = time.time()
    '''生成剩余下法的胜利初始化表'''
    s, (v) = ox.board_now()
    move_value = {x:0 for x in v}
    #'''一个产生下法序列的生成器函数，穷举直到不再有新的下法序列'''
    #'''使用了排列生成，生成函数是个函数生成器； 返回剩余下法的所有排列'''
    mvs = itertools.permutations(v) 
    for mv in mvs:
        move_value[mv[0]] += ttt_simu(ox, playturn, mv)
        
    if playturn == 'o':
        minvalue = move_value[min(move_value, key=move_value.get)]
        mvlist = [k for k,kv in move_value.items() if kv==minvalue ]
    if playturn == 'x':
        maxvalue = move_value[max(move_value, key=move_value.get)]
        mvlist = [k for k,kv in move_value.items() if kv==maxvalue ]

    return mvlist[random.randint(0,len(mvlist)-1)] #返回元组(x,y)

          


In [28]:
'''生成井字棋棋盘，并获得初始状态 ''' 
ttt = Board(3, 3, 3)
ttt_states, (ttt_availables) = ttt.board_now()
player = 'x'
i = 0
while True :
    print("X先手，第%d回合："%i)
    i += 1
    ttt.board_show()

    if player == 'x':
        #print('请选手选择落子处的数字，如23代表在第2行第3列落子：', end='')
        #xy = input()
        #playlocation = (int(xy[0]), int(xy[1]) )
        playlocation = ai_montecarlo_search(ttt, player, 200)
        ttt.board_update(player, playlocation)
        s, (v) = ttt.board_now()
        if s == 1:
            print('选手X 赢')
            ttt.board_show()
            break
        if v == set():
            print('平局')
            ttt.board_show()
            break
        player = 'o'

    if player == 'o':
        playlocation = ai_full_search(ttt, player)
        #playlocation = ai_montecarlo_search(ttt, player, 1000)
        ttt.board_update(player, playlocation)
        s, (v) = ttt.board_now()
        if s == -1:
            print('计算机Q 赢')
            ttt.board_show()
            break 
        if v == set():
            print('平局')
            ttt.board_show()
            break
        player = 'x'


X先手，第0回合：
                      x\y  0    1    2  

                        0  +    +    +  

                        1  +    +    +  

                        2  +    +    +  

X先手，第1回合：
                      x\y  0    1    2  

                        0  X    +    +  

                        1  +    Q    +  

                        2  +    +    +  

X先手，第2回合：
                      x\y  0    1    2  

                        0  X    X    Q  

                        1  +    Q    +  

                        2  +    +    +  

X先手，第3回合：
                      x\y  0    1    2  

                        0  X    X    Q  

                        1  Q    Q    +  

                        2  X    +    +  

X先手，第4回合：
                      x\y  0    1    2  

                        0  X    X    Q  

                        1  Q    Q    X  

                        2  X    Q    +  

平局
                      x\y  0    1    2  

                        0  X    X    Q  

                       